In [3]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import zipfile
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

# 隐藏状态初始化函数

In [10]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (
            _one((num_inputs,num_hiddens)),
            _one((num_hiddens,num_hiddens)),
            torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True)
        )
    W_xz,W_hz,b_z = _three() # 更新门参数
    W_xr,W_hr,b_r = _three() # 重置门参数
    W_xh,W_hh,b_h = _three() # 候选隐藏状态参数
    
    # 输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cpu


In [5]:
def init_gru_state(batch_size, num_hiddens, device):
    # 返回由一个形状为(批量大小, 隐藏单元个数)的值为0的Tensor组成的元组。
    return (torch.zeros((batch_size,num_hiddens), device=device),)

#  定义模型

In [12]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [8]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [13]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 150.196505, time 1.22 sec
 - 分开 我想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 
 - 不分开 我想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 你想你的让我 
epoch 80, perplexity 32.879551, time 1.43 sec
 - 分开 一直我 你想我 你你的我有你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
 - 不分开  没有你在我有多难难熬  没有你想我想要你的怒火 你想想这样 我不要再想 我不要 我不要 我不要再
epoch 120, perplexity 5.997008, time 1.31 sec
 - 分开 一场风 是你的美空在 老彻底 全又了 分数的脚丽 老辛无  全了了双截棍 哼哼哈兮 快使用双截棍 
 - 不分开 你是我面开睡猜 我想要你的微笑每天都能看到  我知道这里很美但 一定到最只妈 你在那里 在小村外的
epoch 160, perplexity 1.775374, time 1.24 sec
 - 分开 这小的传猫你眼单 从因你手了泪被野般 脸上汹涌失控的母言 一切一口吴侬 你的完忆主义 太彻底著出球
 - 不分开 你是我不开睡汉猜 看后走的让你知道 爱情你没爱你来单 说因你说 你我的外婆家 一起看着日落 一直到


# 简洁实现

In [14]:
lr = 1e-2 # 注意，这里使用torch的自带api，所以使用正常的学习率
gru_layer = nn.GRU(input_size=vocab_size,hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer,vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.016433, time 1.05 sec
 - 分开球 我说的话 你甘会听 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切 让家庭回到过
 - 不分开始打我 三分球 它在空中停留 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅
epoch 80, perplexity 1.011070, time 1.06 sec
 - 分开的话 在远方决斗 在一处被废弃的白蛦丘 站着一只饿昏的老斑鸠 印地安老斑鸠 腿短毛不多 几天都没有喝
 - 不分开始打我 别人的梦 全面放纵 恨没有用 疗伤止痛 不再感动 没有梦 痛不知轻重 泪水鲜红 全面放纵 恨
epoch 120, perplexity 1.010341, time 1.05 sec
 - 分开的黑色幽默 不想太多 我想一定是我听错弄错搞错 拜托 我想是你的脑袋有问题 随便说说 其实我早已经猜
 - 不分开始的美丽 纪录第一次遇见的你 如果我遇见你是一场悲剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的
epoch 160, perplexity 1.011865, time 1.07 sec
 - 分开的爸 在远方决斗 在一处被废弃的白蛦丘 站着一只饿昏的老斑鸠 印地安老斑鸠 腿短毛不多 几天都没有喝
 - 不分开  心碎哭泣 再狠狠忘记 你爱过我的证据 让晶莹的泪滴 闪烁成回忆 伤人的美丽 你的完美主义 太彻底
